In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import json

In [2]:
# For testing of code, remove this afterwards, and add in the line to receive the processed data
raw_train2 = pd.read_csv('train_data.csv')
raw_train = pd.read_csv('train.csv')
raw_test = pd.read_csv('test.csv')

In [6]:
raw_train

,Gender,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,ChurnDate,TransactionFreq,TransactionAmt,ServiceSupportFrequency,...,CurrLifecycle,Happiness,Savings Savant,Digital Dynamos,Trustee Tribe,FirstPersona,SecondPersona,ThirdPersona,SocialInfluencer,CombinedPersonas
0,Male,38,15,158584.28,1,76640.29,NaN,15.092344,145925.493884,8,...,Active,0,28.508287,24.198895,47.292818,Trustee Tribe,NaN,NaN,0,Trustee Tribe
1,Female,36,3,94216.05,1,179128.69,2024-03-28,9.192966,73541.740488,35,...,Reactivated,0,14.342629,40.398406,45.258964,Trustee Tribe,Digital Dynamos,NaN,0,Premium Patrons
2,Female,35,11,0.00,2,19842.91,NaN,29.023334,68936.936250,7,...,Active,0,41.009464,24.290221,34.700315,Savings Savant,Trustee Tribe,NaN,1,Cost-Conscious Careseekers
3,Female,26,11,145980.23,1,131804.86,2023-09-08,29.562389,53097.327051,3,...,Active,0,35.205365,38.977368,25.817267,Digital Dynamos,Savings Savant,NaN,0,Frugal Innovators
4,Male,34,1,0.00,2,10063.75,NaN,53.452905,4434.315367,8,...,Active,0,29.319781,36.591087,34.089132,Digital Dynamos,Trustee Tribe,NaN,0,Premium Patrons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33001,Male,45,11,104576.80,1,113081.42,NaN,49.134857,64340.400480,24,...,Reactivated,0,32.868526,28.685259,38.446215,Trustee Tribe,Savings Savant,NaN,0,Cost-Conscious Careseekers
33002,Male,43,13,0.00,2,88866.39,NaN,14.045808,4432.861455,8,...,Active,0,34.285714,39.047619,26.666667,Digital Dynamos,Savings Savant,NaN,0,Frugal Innovators
33003,Female,39,13,0.00,2,167973.63,NaN,7.608855,65313.045274,10,...,Active,0,35.311005,37.320574,27.368421,Digital Dynamos,Savings Savant,NaN,0,Frugal Innovators
33004,Female,48,7,0.00,2,24998.75,NaN,28.199759,54917.137710,32,...,Reactivated,0,39.726027,24.657534,35.616438,Savings Savant,Trustee Tribe,NaN,0,Cost-Conscious Careseekers


In [5]:
cols = raw_train.columns
X = raw_train[cols[3:-1]].drop(columns=['Gender', 'Geography'])
y = raw_train[
[cols[-1]]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyError: "['Gender', 'Geography'] not found in axis"

In [43]:
## CODE TO PREPROCESS THE DATASET

cols = raw_train.columns
#X = raw_train.iloc[:,3:].drop(columns=['Gender', 'Geography'])
X = raw_train.drop(columns=['Gender', 'ChurnDate', 'FirstPersona', 'SecondPersona', 'ThirdPersona', 'CombinedPersonas', 'SocialInfluencer'])
X['Active'] = (X['CurrLifecycle'] == 'Active').astype(int)
X['Reactivated'] = (X['CurrLifecycle'] == 'Reactivated').astype(int)
X['Dormant'] = (X['CurrLifecycle'] == 'Dormant').astype(int)
X['Churned'] = (X['CurrLifecycle'] == 'Churned').astype(int)

strat = raw_train['CurrLifecycle']
all_train, all_test, strat_train, strat_test = train_test_split(X, strat, test_size=0.2, random_state=42)

In [ ]:
#y_train_a, y_train_r, y_train_d, y_train_c = all_train['Active'], all_train['Reactivated'], all_train['Dormant'], all_train['Churned']
#y_test_a, y_test_r, y_test_d, y_test_c = all_test['Active'], all_test['Reactivated'], all_test['Dormant'], all_test['Churned']

y_train_all = [all_train['Active'], all_train['Reactivated'], all_train['Dormant'], all_train['Churned']]
y_test_all = [all_test['Active'], all_test['Reactivated'], all_test['Dormant'], all_test['Churned']]
X_train = all_train.drop(columns=['Active', 'Reactivated', 'Dormant', 'Churned'])
X_test = all_test.drop(columns=['Active', 'Reactivated', 'Dormant', 'Churned'])

In [ ]:
## WONT USE THIS
y_train_a, y_train_r, y_train_d, y_train_c = y_train, y_train, y_train, y_train
y_test_a, y_test_r, y_test_d, y_test_c = y_test, y_test, y_test, y_test

y_train_all = [y_train_a, y_train_r, y_train_d, y_train_c]
y_test_all = [y_test_a, y_test_r, y_test_d, y_test_c]

In [56]:
#Code to ignore the DataConversionWarning
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=DataConversionWarning)

## Logistic Regression

In [57]:
lr_model = LogisticRegression()

In [58]:
# Hyperparameter Tuning
lr_param_dist = {
    'penalty': ['l1', 'l2'],  
    #'C' : np.logspace(-4, 4, 20),
    'C': [0.1, 1, 10, 100, 1000],  
    'solver': ['liblinear', 'saga'] 
}

#search_lr = RandomizedSearchCV(lr_model, param_distributions=lr_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)

# Scale the data for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform optimization
#search_lr.fit(X_train_scaled, y_train)

#print("Best parameters found: ", search_lr.best_params_)

# Evaluate the best model on the test set
#best_model_lr = search_lr.best_estimator_
#lr_predictions = best_model_lr.predict(X_test_scaled)
#lr_probabilities = best_model_lr.predict_proba(X_test_scaled)[:,1]
#test_score_lr = best_model_lr.score(X_test_scaled, y_test)
#print("Test score of the best model: ", test_score_lr)


In [59]:
lr_models = []
lr_results = []

for i in range(4):
    lr_model = LogisticRegression()
    search_lr = RandomizedSearchCV(lr_model, param_distributions=lr_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)

    search_lr.fit(X_train_scaled, y_train_all[i])

    best_model_lr = search_lr.best_estimator_
    lr_models.append(best_model_lr)
    
    test_score_lr = best_model_lr.score(X_test_scaled, y_test_all[i])
    lr_results.append(test_score_lr)

In [60]:
lr_results

[0.8233405035295543,
 0.8233405035295543,
 0.8233405035295543,
 0.8233405035295543]

## Gradient Booster

In [ ]:
gb_model = GradientBoostingClassifier()

In [ ]:
# Hyperparameter Tuning
gb_param_dist = {
    'loss': ['log_loss', 'exponential'],
    'learning_rate': np.arange(0., 2., 0.2).tolist(),
    'n_estimators': np.arange(100, 301, 100).tolist(),
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_leaf': np.arange(6,15, 2).tolist(),
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': np.arange(2,15, 2).tolist(),
    'max_depth': np.arange(3,16, 3).tolist()
}

#search = BayesSearchCV(
#    estimator=gb_model,
#    search_spaces=gb_param_dist,
#    n_iter=50,
#    cv=5
#)

search = RandomizedSearchCV(gb_model, param_distributions=gb_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)


# Perform optimization
search.fit(X_train, y_train)

print("Best parameters found: ", search.best_params_)

# Evaluate the best model on the test set
best_model = search.best_estimator_
gb_predictions = best_model.predict(X_test)
test_score = best_model.score(X_test, y_test)
print("Test score of the best model: ", test_score)


In [75]:
xgb_model = xgb.XGBClassifier()

In [76]:
# Define the XGBoost parameter ranges
xgb_param_dist = {
    'booster': ['gbtree'],  
    'learning_rate': np.linspace(0.05, 0.3, 6),  
    'n_estimators': [100, 200],  
    'objective': ['multi:softmax'],  
    'num_class': [2],  
    'eval_metric': ['logloss'],  
    'max_depth': [3, 6, 9],  
    'min_child_weight': [1, 5, 10],  
    'gamma': [0, 0.1, 0.2],  
    'subsample': [0.6, 0.8, 1.0],  
    'colsample_bytree': [0.6, 0.8, 1.0],  
    'lambda': [0, 1, 2],  
    'alpha': [0, 1, 2]  
}

xgb_param_dist2 = {
    'booster': ['gbtree', 'dart'],  
    'learning_rate': np.linspace(0.05, 0.3, 6),  
    'n_estimators': [100, 200, 300],  
    'objective': ['binary:logistic', 'multi:softmax'],  
    'num_class': [2],
    'eval_metric': ['logloss'],  
    'max_depth': [3, 6, 9, 12],  
    'min_child_weight': [1, 5, 10],  
    'gamma': [0, 0.1, 0.2],  
    'subsample': [0.6, 0.8, 1.0],  
    'colsample_bytree': [0.6, 0.8],  
    'lambda': [0, 1, 2],  
    'alpha': [0, 1, 2]  
}

# Instantiate RandomizedSearchCV
#xgb_search = RandomizedSearchCV(xgb_model, param_distributions=xgb_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)

# Perform optimization
#xgb_search.fit(X_train, y_train)

#print("Best parameters found: ", xgb_search.best_params_)

# Evaluate the best model on the test set
#best_xgb_model = xgb_search.best_estimator_
#xgb_predictions = best_xgb_model.predict(X_test)
#test_score = best_xgb_model.score(X_test, y_test)
#print("Test score of the best XGBoost model: ", test_score)

In [78]:
xgb_models = []
xgb_results = []

for i in range(1):
    xgb_model = xgb.XGBClassifier()
    xgb_search = RandomizedSearchCV(xgb_model, param_distributions=xgb_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)

    xgb_search.fit(X_train, y_train_all[i])

    best_model_xgb = xgb_search.best_estimator_
    xgb_models.append(best_model_xgb)
    
    test_score_xgb = best_model_xgb.score(X_test, y_test_all[i])
    xgb_results.append(test_score_xgb)

In [87]:
xgb_models

[XGBClassifier(alpha=2, base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=0.1, grow_policy=None,
               importance_type=None, interaction_constraints=None, lambda=2,
               learning_rate=0.1, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
               max_leaves=None, min_child_weight=10, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=100,
               n_jobs=None, ...),
 XGBClassifier(alpha=2, base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False

## Random Forest

In [97]:
rf_model = RandomForestClassifier()

In [94]:
rf_param_dist = {
    'n_estimators': [100],
    'criterion': ['gini'],
    'max_depth': [None] + list(range(5, 11, 5)),
    'min_samples_leaf': [1, 5],
    'min_samples_split': [2, 10]
}
#search = BayesSearchCV(
    #estimator=rf_model,
    #search_spaces=rf_param_dist,
    #n_iter=50,
    #cv=5
#)
#search = RandomizedSearchCV(rf_model, param_distributions=rf_param_dist, n_iter=10, scoring='f1_weighted', cv=5, random_state=42)

# Perform optimization
#search.fit(X_train, y_train)

#print("Best parameters found: ", search.best_params_)

# Evaluate the best model on the test set
#best_model = search.best_estimator_
#rf_predictions = best_model.predict(X_test)
#test_score = best_model.score(X_test, y_test)
#print("Test score of the best model: ", test_score)

In [96]:
rf_models = []
rf_results = []

for i in range(1):
    rf_model = RandomForestClassifier()
    #search_rf = RandomizedSearchCV(rf_model, param_distributions=rf_param_dist, n_iter=5, scoring='f1_weighted', cv=5, random_state=42)

    #search_rf.fit(X_train, y_train_all[i])

    #best_model_rf = search_rf.best_estimator_
    best_model_rf = rf_model.fit(X_train, y_train_all[i])
    rf_models.append(best_model_rf)
    
    test_score_rf = best_model_rf.score(X_test, y_test_all[i])
    rf_results.append(test_score_rf)

KeyboardInterrupt: 

In [102]:
rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, 
                                  min_samples_leaf=1, 
                                  min_samples_split=2, random_state=42)

In [103]:
best_model_rf = rf_model.fit(X_train, y_train_all[i])

In [110]:
rf_models.append(best_model_rf)

In [113]:
xgb_models

[XGBClassifier(alpha=2, base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=0.1, grow_policy=None,
               importance_type=None, interaction_constraints=None, lambda=2,
               learning_rate=0.1, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
               max_leaves=None, min_child_weight=10, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=100,
               n_jobs=None, ...),
 XGBClassifier(alpha=2, base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False

## Send Code to The Next Person

In [64]:
lr_models

[LogisticRegression(C=1000, penalty='l1', solver='saga'),
 LogisticRegression(C=10, solver='saga'),
 LogisticRegression(C=10, penalty='l1', solver='liblinear'),
 LogisticRegression(C=1, penalty='l1', solver='saga')]

In [117]:
lr_model_dicts = []
for model in lr_models:
    model_dict = {
        "C": model.C,
        "penalty": model.penalty,
        "solver": model.solver
        # Add more attributes as needed
    }
    lr_model_dicts.append(model_dict)

rf_model_dicts = []
for model in rf_models:
    model_dict = {
        'n_estimators': model.n_estimators,
        'criterion': model.criterion,
        'max_depth': model.max_depth,
        'min_samples_leaf': model.min_samples_leaf,
        #'max_features': model.max_features,
        'min_samples_split': model.min_samples_split
    }
    rf_model_dicts.append(model_dict)

xgb_model_dicts = []
for model in xgb_models:
    model_dict = {
        'booster': model.booster,  
        'learning_rate': model.learning_rate,  
        'n_estimators': model.n_estimators,  
        'objective': model.objective,  
        #'num_class': model.num_class,  
        'eval_metric': model.eval_metric,  
        'max_depth': model.max_depth,  
        'min_child_weight': model.min_child_weight,  
        'gamma': model.gamma,  
        'subsample': model.subsample,  
        'colsample_bytree': model.colsample_bytree,  
        'lambda': getattr(model, 'lambda', None),  # Access lambda attribute using getattr()
        'alpha': getattr(model, 'alpha', None) 
    }
    if hasattr(model, 'num_class'):
            model_dict['num_class'] = model.num_class
    xgb_model_dicts.append(model_dict)

In [118]:
models = [lr_model_dicts, xgb_model_dicts, rf_model_dicts]

In [119]:
models

[[{'C': 1000, 'penalty': 'l1', 'solver': 'saga'},
  {'C': 10, 'penalty': 'l2', 'solver': 'saga'},
  {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'},
  {'C': 1, 'penalty': 'l1', 'solver': 'saga'}],
 [{'booster': 'gbtree',
   'learning_rate': 0.1,
   'n_estimators': 100,
   'objective': 'multi:softmax',
   'eval_metric': 'logloss',
   'max_depth': 6,
   'min_child_weight': 10,
   'gamma': 0.1,
   'subsample': 1.0,
   'colsample_bytree': 0.8,
   'lambda': None,
   'alpha': None},
  {'booster': 'gbtree',
   'learning_rate': 0.1,
   'n_estimators': 100,
   'objective': 'multi:softmax',
   'eval_metric': 'logloss',
   'max_depth': 6,
   'min_child_weight': 10,
   'gamma': 0.1,
   'subsample': 1.0,
   'colsample_bytree': 0.8,
   'lambda': None,
   'alpha': None},
  {'booster': 'gbtree',
   'learning_rate': 0.1,
   'n_estimators': 100,
   'objective': 'multi:softmax',
   'eval_metric': 'logloss',
   'max_depth': 6,
   'min_child_weight': 10,
   'gamma': 0.1,
   'subsample': 1.0,
   'colsampl

In [120]:
models = [lr_model_dicts, xgb_model_dicts, rf_model_dicts]
#models = [lr_models, xgb_models, rf_models]

# Convert the list to JSON
json_data = json.dumps(models, indent=4)

# Write JSON data to a file
with open("Models.json", "w") as json_file:
    json_file.write(json_data)

In [7]:
features = ['Gender', 'ChurnDate', 'FirstPersona', 'SecondPersona', 'ThirdPersona', 'CombinedPersonas', 'CurrLifecycle','SocialInfluencer',
           'CurrLifecycle_Active','CurrLifecycle_Reactivated','CurrLifecycle_Churned','CurrLifecycle_Dormant',
           'PrevLifecycle_Active','PrevLifecycle_Churned','PrevLifecycle_Dormant','PrevLifecycle_Reactivated']

# Convert the list to JSON
json_data = json.dumps(features)

# Write JSON data to a file
with open("FeatureDropped.json", "w") as json_file:
    json_file.write(json_data)